In [6]:
import numpy as np
import trimesh
import networkx as nx
from sklearn.metrics.pairwise import haversine_distances
from sklearn.neighbors import BallTree
from scipy.spatial.transform import Rotation as R
import xarray as xr
from sklearn.neighbors import BallTree

In [7]:
def direction_vec(v1, v2, epsilon=10e-11):
    v = np.cross(v1, v2)
    vnorm1 = np.dot(v, v)
    if vnorm1 < epsilon:
        v1 = v1 + epsilon
        v = np.cross(v1, v2)
        vnorm1 = np.dot(v, v)
    return v / np.sqrt(vnorm1)

In [8]:
def create_sphere(subdivisions=0, radius=1.0):
    return trimesh.creation.icosphere(subdivisions=subdivisions, radius=radius)

In [9]:
def to_unit_sphere_xyz(loc):
    latr, lonr = loc
    R = 1.0
    x = R * np.cos(latr) * np.cos(lonr)
    y = R * np.cos(latr) * np.sin(lonr)
    z = R * np.sin(latr)
    return np.array((x, y, z))

In [10]:
def get_rotation_from_unit_vecs(v1, v2):
    v_unit = direction_vec(v1, v2)
    theta = np.arccos(np.dot(v1, v2))
    return R.from_rotvec(v_unit * theta)

In [11]:
def to_latlon(xyz, radius=1.0):
    lat = np.arcsin(xyz[..., 2] / radius) * 180.0 / np.pi
    lon = np.arctan2(xyz[..., 1], xyz[..., 0]) * 180.0 / np.pi
    return np.array((lat, lon), dtype=np.float32).transpose()

In [12]:
def to_rad(xyz, radius=1.0):
    lat = np.arcsin(xyz[..., 2] / radius)
    lon = np.arctan2(xyz[..., 1], xyz[..., 0])
    return np.array((lat, lon), dtype=np.float32).transpose()

In [61]:
to_rad(np.array([34,84,0]))

array([0.       , 1.1861916], dtype=float32)

In [13]:
def compute_directions(loc1, loc2, pole_vec=(0.0, 0.0, 1.0)):
    pole_vec = np.array(pole_vec)  # all will be rotated relative to destination node
    loc1_xyz = to_unit_sphere_xyz(loc1)
    loc2_xyz = to_unit_sphere_xyz(loc2)
    r = get_rotation_from_unit_vecs(loc2_xyz, pole_vec)  # r.apply(loc1_xyz) == pole_vec
    direction = direction_vec(r.apply(loc1_xyz), pole_vec)
    return direction / np.sqrt(np.dot(direction, direction))

In [14]:
def directional_edge_features_rotated(loc1, loc2):
    return compute_directions(loc1, loc2)[0:2]  # discard last component -> zero if rotated to north pole

In [15]:
def get_one_ring(sp):
    g = nx.from_edgelist(sp.edges_unique)
    one_ring = [list(g[i].keys()) for i in range(len(sp.vertices))]

    return one_ring

In [16]:
def add_edge(G, idx1, idx2, allow_self_loop=False, add_edge_attrib=False):
    loc1 = np.deg2rad(h3.h3_to_geo(idx1))
    loc2 = np.deg2rad(h3.h3_to_geo(idx2))
    if allow_self_loop or idx1 != idx2:
        if add_edge_attrib:
            direction = directional_edge_features_rotated(loc1, loc2)
            G.add_edge(
                idx1,
                idx2,
                edge_attr=(haversine_distances([loc1, loc2])[0][1], *direction),
            )

        else:
            G.add_edge(idx1, idx2, weight=haversine_distances([loc1, loc2])[0][1])
            # G.add_edge(idx1, idx2, weight = h3.point_dist(loc1, loc2, unit='rads'))


In [17]:
def add_nodes(G, resolution, self_loop=False):
    for idx in h3.uncompact(h3.get_res0_indexes(), resolution):
        G.add_node(idx, hcoords_rad=np.deg2rad(h3.h3_to_geo(idx)))
        if self_loop:
            add_edge(G, idx, idx, allow_self_loop=self_loop)

In [18]:
def multi_mesh2(resolutions):
    G = nx.Graph()

    sp1 = create_sphere(resolutions[-1])
    sp1_rad = to_rad(sp1.vertices)
    one_rings = get_one_ring(sp1)

    for ii, coords in enumerate(sp1_rad):
        G.add_node(ii, hcoords_rad=sp1_rad[ii])

    for ii in range(len(sp1.vertices)):
        for ineighb in one_rings[ii]:
            if ineighb != ii:
                loc_self = sp1_rad[ii]
                loc_neigh = sp1_rad[ineighb]
                # direction = directional_edge_features_rotated(loc_neigh, loc_self)
                # G.add_edge(ineighb, ii, edge_attr=(haversine_distances([loc_neigh, loc_self])[0][1], *direction))
                G.add_edge(ineighb, ii, weight=haversine_distances([loc_neigh, loc_self])[0][1])

    tree = BallTree(sp1_rad, metric="haversine")

    for resolution in resolutions[:-1]:
        sp2 = create_sphere(resolution)
        sp2_rad = to_rad(sp2.vertices)
        one_rings = get_one_ring(sp2)

        dist, ind1 = tree.query(sp2_rad, k=1)
        for ii in range(len(sp2.vertices)):
            for ineighb in one_rings[ii]:
                if ineighb != ii:
                    loc_dst = sp2_rad[ii]
                    loc_neigh = sp2_rad[ineighb]
                    # direction = directional_edge_features_rotated(loc_neigh, loc_dst)
                    # G.add_edge(ind1[ineighb][0], ind1[ii][0], \
                    # edge_attr=(haversine_distances([loc_neigh, loc_dst])[0][1], *direction))
                    G.add_edge(ind1[ineighb][0], ind1[ii][0], weight=haversine_distances([loc_neigh, loc_dst])[0][1])

    return G, sp1_rad

In [20]:
def plot_graph_from_networkx(title, H3):
    # largely from https://plotly.com/python/network-graphs/
    import plotly.graph_objects as go

    edge_x = []
    edge_y = []
    for edge in H3.edges():
        x0, y0 = np.rad2deg(H3.nodes[edge[0]]["hcoords_rad"])
        x1, y1 = np.rad2deg(H3.nodes[edge[1]]["hcoords_rad"])
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scattergeo(lat=edge_x, lon=edge_y, line=dict(width=0.5, color="#888"), hoverinfo="none", mode="lines")

    node_x = []
    node_y = []
    for node in H3.nodes():
        x, y = np.rad2deg(H3.nodes[node]["hcoords_rad"])
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scattergeo(
        lat=node_x,
        lon=node_y,
        mode="markers",
        hoverinfo="text",
        marker=dict(
            showscale=True,
            colorscale="YlGnBu",
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(thickness=15, title="Node Connections", xanchor="left", titleside="right"),
            line_width=2,
        ),
    )

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(H3.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append("# of connections: " + str(len(adjacencies[1])))

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            title="<br>" + title,
            titlefont_size=16,
            showlegend=False,
            hovermode="closest",
            margin=dict(b=20, l=5, r=5, t=40),
            annotations=[
                dict(
                    text="",
                    showarrow=False,
                    xref="paper",
                    yref="paper",
                    x=0.005,
                    y=-0.002,
                )
            ],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        ),
    )
    fig.show()
    n_h3_nodes0 = 122
    n_h3_nodes1 = 842
    print(sorted(node_adjacencies, reverse=True)[0:n_h3_nodes0])
    print(sorted(node_adjacencies, reverse=True)[n_h3_nodes0 : n_h3_nodes0 + n_h3_nodes1])
    print(sorted(node_adjacencies, reverse=True)[n_h3_nodes0 + n_h3_nodes1 : n_h3_nodes0 + n_h3_nodes1 + 100])


In [202]:
init_mesh = multi_mesh2([0])

In [203]:
plot_graph_from_networkx("initial mesh", init_mesh[0])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
[]
[]


In [116]:
def get_triangle_nodes(mesh, data):
    lat = data['lat'].to_numpy()
    lon = data['lon'].to_numpy()
    triangle_nodes = []
    tree = BallTree(mesh[1], metric="haversine")
    # for i in range(0, np.shape(lat)[0]-1, 40):
    #     for j in range(0, np.shape(lat)[1]-1, 40):
    for i in range(0, 40, 1):
        for j in range(0, 40, 1):
            coords = to_unit_sphere_xyz([lat[i,j], lon[i,j]])
            nlat, nlon = to_rad(coords)
            node = tree.query([[nlat, nlon]])[1][0][0]
            # print("lat", lat[i,j])
            # print("lon", lon[i,j])
            # print(nlat, nlon)
            if node not in triangle_nodes:
                triangle_nodes.append(int(node))
    return triangle_nodes

In [120]:
print(type(init_mesh[0]))

<class 'networkx.classes.graph.Graph'>


In [37]:
data2 = xr.open_dataset("../../../../../../nobackup/users/buurmans/datatest.nc", decode_coords = "all")

In [190]:
def _grid_lat_lon_to_coordinates(
    grid_latitude: np.ndarray, grid_longitude: np.ndarray) -> np.ndarray:
    """Lat [num_lat] lon [num_lon] to 3d coordinates [num_lat, num_lon, 3]."""
    # Convert to spherical coordinates phi and theta defined in the grid.
    # Each [num_latitude_points, num_longitude_points]
    phi_grid, theta_grid = np.meshgrid(
      np.deg2rad(grid_longitude),
      np.deg2rad(90 - grid_latitude))

    # [num_latitude_points, num_longitude_points, 3]
    # Note this assumes unit radius, since for now we model the earth as a
    # sphere of unit radius, and keep any vertical dimension as a regular grid.
    return np.stack(
      [np.cos(phi_grid)*np.sin(theta_grid),
       np.sin(phi_grid)*np.sin(theta_grid),
       np.cos(theta_grid)], axis=-1)

In [191]:
def get_triangle_faces(mesh, data):
    lat = data['lat'].to_numpy()
    lon = data['lon'].to_numpy()
    triangle_faces = []
    mesh = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
    for i in range(0, np.shape(lat)[0]-1, 40):
        for j in range(0, np.shape(lat)[1]-1, 40):
            _, _, triangle = trimesh.proximity.closest_point(mesh, _grid_lat_lon_to_coordinates(lat[i,j], lon[i,j])[0])
            if triangle not in triangle_faces:
                triangle_faces.append(int(triangle))
    return triangle_faces

In [197]:
most_refined = create_sphere(3)
tni = get_triangle_faces(most_refined, data2)

In [198]:
len(tni)

22

In [194]:
def multi_mesh_local(resolutions, local_faces):
    G = nx.Graph()
    mesh_trimesh= create_sphere(resolutions[-1])
    # subdivide the mesh for the local faces
    sp1 = mesh_trimesh.subdivide(local_faces)
    
    # project to the unit sphere
    vectors = sp1.vertices
    scalar = np.sqrt(np.dot(vectors**2, [1, 1, 1]))
    unit = vectors / scalar.reshape((-1, 1))
    sp1.vertices += unit * (1 - scalar).reshape((-1, 1))

    # to radial coordinates
    sp1_rad = to_rad(sp1.vertices)
    one_rings = get_one_ring(sp1)

    for ii, coords in enumerate(sp1_rad):
        G.add_node(ii, hcoords_rad=sp1_rad[ii])

    for ii in range(len(sp1.vertices)):
        for ineighb in one_rings[ii]:
            if ineighb != ii:
                loc_self = sp1_rad[ii]
                loc_neigh = sp1_rad[ineighb]
                # direction = directional_edge_features_rotated(loc_neigh, loc_self)
                # G.add_edge(ineighb, ii, edge_attr=(haversine_distances([loc_neigh, loc_self])[0][1], *direction))
                G.add_edge(ineighb, ii, weight=haversine_distances([loc_neigh, loc_self])[0][1])

    tree = BallTree(sp1_rad, metric="haversine")

    for resolution in resolutions[:-1]:
        sp2 = create_sphere(resolution)
        sp2_rad = to_rad(sp2.vertices)
        one_rings = get_one_ring(sp2)

        dist, ind1 = tree.query(sp2_rad, k=1)
        for ii in range(len(sp2.vertices)):
            for ineighb in one_rings[ii]:
                if ineighb != ii:
                    loc_dst = sp2_rad[ii]
                    loc_neigh = sp2_rad[ineighb]
                    # direction = directional_edge_features_rotated(loc_neigh, loc_dst)
                    # G.add_edge(ind1[ineighb][0], ind1[ii][0], \
                    # edge_attr=(haversine_distances([loc_neigh, loc_dst])[0][1], *direction))
                    G.add_edge(ind1[ineighb][0], ind1[ii][0], weight=haversine_distances([loc_neigh, loc_dst])[0][1])
        

    return G, sp1_rad

In [200]:
test_mesh = multi_mesh_local([0,1,2,3], tni)

In [149]:
test_mesh[0].nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47))

In [150]:
init_mesh[0].nodes

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41))

In [201]:
plot_graph_from_networkx("test mesh", test_mesh[0])

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 14, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,